In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA, KernelPCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score, classification_report
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, StratifiedKFold, train_test_split
from sklearn.feature_selection import VarianceThreshold, RFE, SelectKBest, chi2
from sklearn.ensemble import BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, VotingClassifier, RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


Creating a baseline Regression model for better analysis 

In [0]:
train_df = pd.read_csv('training.csv')
test_df = pd.read_csv('test.csv')
target_df = pd.read_csv('Training set Labels.csv')

In [0]:
target_df['status_group'] = target_df['status_group'].replace({'functional':0, 'non functional':1, 'functional needs repair':2})

In [4]:
target_df.head(20)

,id,status_group
0,69572,0
1,8776,0
2,34310,0
3,67743,1
4,19728,0
5,9944,0
6,19816,1
7,54551,1
8,53934,1
9,46144,0


In [0]:
def conversion(df):
  for cols in df.columns:
    df[cols] = pd.to_numeric(df[cols])
  return df

In [0]:
train_df = conversion(train_df)
test_df = conversion(test_df)

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Columns: 186 entries, id to waterpoint_type#other
dtypes: float64(4), int64(182)
memory usage: 84.3 MB


Standardizing the numerical variables

In [0]:
numer_feat = ['amount_tsh','gps_height','population','construction_year']
scaler =  MinMaxScaler()
train_df[numer_feat] = scaler.fit_transform(train_df[numer_feat])
test_df[numer_feat]  = scaler.transform(test_df[numer_feat])

In [0]:
from keras.utils import to_categorical
target_copy=target_df.copy()
target_copy = target_copy.drop('id',axis=1)

In [0]:
y = target_copy['status_group'].values
y =  to_categorical(y)

In [0]:
X = train_df.loc[:,:].values

In [0]:
#Splitting the train and test set 
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0,test_size=0.25)

In [13]:
X_train.shape , X_test.shape , len(y_train) , len(y_test)

((44550, 186), (14850, 186), 44550, 14850)

In [0]:
splits= 5
scoring = 'accuracy'
seed=0
kfold = KFold(n_splits=splits, shuffle=True, random_state=0)

**Random Forest Classifier**

In [0]:
rfc =RandomForestClassifier(n_estimators=1000, 
                            min_samples_split=6,
                            criterion='gini', 
                            max_features='auto',
                            oob_score=True,
                            random_state=1,
                            n_jobs=-1)

In [16]:
rfc.fit(X_train, y_train)
print('Random Forest Classifier Train Accuracy Score :', rfc.score(X_train, y_train))
print('Random Forest Classifier Test Score :', rfc.score(X_test, y_test))

Random Forest Classifier Train Accuracy Score : 0.9092255892255893
Random Forest Classifier Test Score : 0.7781144781144781


In [26]:
predicted = rfc.predict(X_test)
report = classification_report(y_test, predicted)
print(report)

              precision    recall  f1-score   support

           0       0.81      0.88      0.84      8022
           1       0.87      0.74      0.80      5717
           2       0.67      0.24      0.35      1111

   micro avg       0.83      0.78      0.80     14850
   macro avg       0.78      0.62      0.67     14850
weighted avg       0.82      0.78      0.79     14850
 samples avg       0.78      0.78      0.78     14850



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Extra Tree Classifier**

In [17]:
num_trees = 1000
num_features = 10

et_model = ExtraTreesClassifier(n_estimators=num_trees, max_features=num_features)
et_model.fit(X_train,y_train)
print("The Extra Tree Classifier Train Accuracy Score:",et_model.score(X_train,y_train))
print('The Extra Tree Classifier Test Score :', et_model.score(X_test, y_test))

The Extra Tree Classifier Train Accuracy Score: 1.0
The Extra Tree Classifier Test Score : 0.768080808080808


In [19]:
predicted = et_model.predict(X_test)
report = classification_report(y_test, predicted)
print(report)

              precision    recall  f1-score   support

           0       0.81      0.83      0.82      8022
           1       0.81      0.76      0.79      5717
           2       0.49      0.34      0.40      1111

   micro avg       0.79      0.77      0.78     14850
   macro avg       0.70      0.64      0.67     14850
weighted avg       0.79      0.77      0.78     14850
 samples avg       0.77      0.77      0.77     14850



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Neural Network**

In [0]:
def pump_baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=186, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [0]:
estimator = KerasClassifier(build_fn=pump_baseline_model, epochs=1000, batch_size=5, verbose=0)

In [17]:
model = pump_baseline_model()
model.fit(x=X_train, y=y_train,epochs=100) 

Epoch 1/1000
44550/44550 [==============================] - 2s 36us/step - loss: 46.2678 - accuracy: 0.4968
Epoch 2/1000
44550/44550 [==============================] - 2s 34us/step - loss: 3.2545 - accuracy: 0.5497
Epoch 3/1000
44550/44550 [==============================] - 2s 35us/step - loss: 2.8170 - accuracy: 0.5836
Epoch 4/1000
44550/44550 [==============================] - 2s 34us/step - loss: 2.7088 - accuracy: 0.5903
Epoch 5/1000
44550/44550 [==============================] - 2s 34us/step - loss: 2.6899 - accuracy: 0.6022
Epoch 6/1000
44550/44550 [==============================] - 2s 34us/step - loss: 2.6014 - accuracy: 0.6139
Epoch 7/1000
44550/44550 [==============================] - 2s 34us/step - loss: 2.6291 - accuracy: 0.6121
Epoch 8/1000
44550/44550 [==============================] - 2s 34us/step - loss: 2.8180 - accuracy: 0.6137
Epoch 9/1000
44550/44550 [==============================] - 2s 34us/step - loss: 2.1982 - accuracy: 0.6374
Epoch 10/1000
44550/44550 [=========

In [18]:
final_results =  model.evaluate(X_test,y_test)
print(final_results)

14850/14850 [==============================] - 0s 19us/step
[0.6799121217213897, 0.7316498160362244]


**Predictions**

In [0]:
predict = et_model.predict(test_df)

In [0]:
submission_df = pd.read_csv('SubmissionFormat.csv')

In [0]:
Final= pd.concat([test_df, submission_df], axis=1)

In [26]:
Final['status_group'] = predict
Final.head(5)

,id,amount_tsh,gps_height,district_code,population,construction_year,year,month,day,funder#District Council,funder#Kkkt,funder#Tasaf,funder#Unicef,funder#World Vision,funder#danida,funder#gov,funder#hesawa,funder#other,funder#rwssp,funder#world_bank,installer#Kkkt,installer#central gov,installer#ces,installer#commu,installer#danida,installer#dwe,installer#gov,installer#hesawa,installer#other,installer#rwe,installer#tcrs,wpt_name#Ofisini,wpt_name#bombani,wpt_name#kanisani,wpt_name#msikitini,wpt_name#other,wpt_name#school,wpt_name#shule ya msingi,wpt_name#shuleni,wpt_name#sokoni,...,management_group#other,management_group#parastatal,management_group#user-group,payment#never pay,payment#other,payment#pay annually,payment#pay monthly,payment#pay per bucket,payment#pay when scheme fails,water_quality#coloured,water_quality#fluoride,water_quality#fluoride abandoned,water_quality#milky,water_quality#salty,water_quality#salty abandoned,water_quality#soft,quantity#dry,quantity#enough,quantity#insufficient,quantity#seasonal,source#dam,source#hand dtw,source#lake,source#machine dbh,source#other,source#rainwater harvesting,source#river,source#shallow well,source#spring,source_class#groundwater,source_class#surface,waterpoint_type#cattle trough,waterpoint_type#communal standpipe,waterpoint_type#communal standpipe multiple,waterpoint_type#dam,waterpoint_type#hand pump,waterpoint_type#improved spring,waterpoint_type#other,id,status_group
0,50785,0.005520,0.729371,3,0.010492,0.981132,2013,2,4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,50785,1.0
1,51630,0.002778,0.580070,2,0.009804,0.754717,2013,2,4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,51630,1.0
2,17168,0.002631,0.579371,2,0.016361,0.943396,2013,2,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,17168,1.0
3,45559,0.000437,0.124825,43,0.008164,0.509434,2013,1,22,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,45559,0.0
4,49871,0.001428,0.472028,3,0.001934,0.754717,2013,3,27,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,49871,1.0


In [0]:
Final['status_group'] = Final['status_group'].replace({0:'functional', 1: 'non functional', 2: 'functional needs repair'})

In [0]:
Final[['id','status_group']].to_csv('Submission.csv', index=False)

In [0]:
sub = pd.read_csv('Submission.csv')

In [0]:
sub = sub.drop('id.1',axis=1)

In [38]:
sub.head(5)

,id,status_group
0,50785,non functional
1,51630,non functional
2,17168,non functional
3,45559,functional
4,49871,non functional


In [39]:
sub.status_group.value_counts()

non functional    8344
functional        6506
Name: status_group, dtype: int64